In [546]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime
import numpy as np
import seaborn as sns

# Import and clean

## Italy

In [547]:
#Adding header entry
file_path_cases = 'data/Italy/italy_labels.csv'

with open(file_path_cases, 'r') as f:
    content = f.readlines()

if content[0][0:5] != 'index':
    content[0] = 'index' + content[0]

with open(file_path_cases, 'w') as f:
    f.writelines(content)
    f.close()

In [548]:
#Cleaning columns
cases_ita_df = pd.read_csv(file_path_cases)
cases_ita_df = cases_ita_df.drop(columns=['index', 'id'])
cases_ita_df = cases_ita_df.iloc[:, :80]

In [549]:
#Fix Bergamo outlier at 2020-05-12 with average of week
cases_ita_df.at[12, '2020-05-12'] = int(cases_ita_df.iloc[12, 72:79].mean())

In [550]:
#Selecting subset of cities by percentile.
#upper_quant_ita = cases_ita_df.iloc[:, 1:].max(axis=1).quantile(0.75)
#cases_ita_df = cases_ita_df[cases_ita_df.iloc[:, 1:].ge(upper_quant_ita).any(axis=1)]

#Selecting subset of cities by number of absoluet cases.
top_30_ita = cases_ita_df.iloc[:, 1:].sum(axis=1).sort_values(ascending=False).head(30).index
cases_ita_df = cases_ita_df.loc[top_30_ita]
selected_cities_ita = cases_ita_df['name'].to_numpy(dtype='str')

## Spain

In [551]:
#Importing CSV
cases_spa_df = pd.read_csv('data/Spain/spain_labels.csv')
cases_spa_df[cases_spa_df.columns[1:]] = cases_spa_df.iloc[:, 1:].astype('int64')

#Outlier due to mis-reporting
cases_spa_df = cases_spa_df.drop(index = 51)

In [552]:
#Selecting subset of cities by quantile
#upper_quant_spa = cases_spa_df.iloc[:, 1:].mean(axis=1).quantile(0.80)
#cases_spa_df = cases_spa_df[cases_spa_df.iloc[:, 1:].ge(upper_quant_spa).any(axis=1)]

#Selecting subset by absolute number of cases.
top_30_spa = cases_spa_df.iloc[:, 1:].sum(axis=1).sort_values(ascending=False).head(30).index
cases_spa_df = cases_spa_df.loc[top_30_spa]
selected_cities_spa = cases_spa_df['name'].to_numpy(dtype='str')

## France

In [553]:
#Importing CSV
cases_fra_df = pd.read_csv('data/France/france_labels.csv')

In [554]:
#Selecting subset of cities by quantiles
#upper_quant_fra = cases_fra_df.iloc[:, 1:].mean(axis=1).quantile(0.8)
#cases_fra_df = cases_fra_df[cases_fra_df.iloc[:, 1:].ge(upper_quant_fra).any(axis=1)]

#Selecting cities by absolute number of cases
top_30_fra = cases_fra_df.iloc[:, 1:].sum(axis=1).sort_values(ascending=False).head(30).index
cases_fra_df = cases_fra_df.loc[top_30_fra]
selected_cities_fra = cases_fra_df['name'].to_numpy(dtype='str')

## England

In [555]:
cases_eng_df = pd.read_csv('data/England/england_labels.csv')

In [556]:
#Selecting cities by quantiles
#upper_quant_eng = cases_eng_df.iloc[:, 1:].mean(axis=1).quantile(0.75)
#cases_eng_df = cases_eng_df[cases_eng_df.iloc[:, 1:].ge(upper_quant_eng).any(axis=1)]

#Selecting subset of cities by absolute value
top_30_eng = cases_eng_df.iloc[:, 1:].sum(axis=1).sort_values(ascending=True).head(30).index
cases_eng_df = cases_eng_df.loc[top_30_eng]
selected_cities_eng = cases_eng_df['name'].to_numpy(dtype='str')

# Aggregating

In [557]:
selected_cities_list = [selected_cities_ita, selected_cities_spa, selected_cities_fra, selected_cities_eng]
cases_dfs_list = [cases_ita_df, cases_spa_df, cases_fra_df, cases_eng_df]
countries = ['ita', 'spa', 'fra', 'eng']
folder_path_dict = {
    'ita' : 'data/Italy/graphs',
    'spa' : 'data/Spain/graphs',
    'fra' : 'data/France/graphs',
    'eng' : 'data/England/graphs',
}
movement_dfs_list = []

In [558]:
for i in range(len(countries)):
    movement_df = pd.DataFrame(columns=['src', 'trg', 'movement', 'date'])
    movement_df = movement_df.astype({'src':'str', 'trg':'str', 'movement':'int64'})
    movement_df['date'] = pd.to_datetime(movement_df['date'])

    folder_path = folder_path_dict[countries[i]]

    for filename in sorted(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as f:
            content = f.readlines()

        if content[0].strip() != ','.join(['src', 'trg', 'movement']):
            content.insert(0, ','.join(['src', 'trg', 'movement']) + '\n')
        
        with open(file_path, 'w') as f:
            f.writelines(content)
            f.close()

        df = pd.read_csv(file_path)
        df = df.astype({'src':'str', 'trg':'str', 'movement':'int64'})
        df = df[(df['src'].isin(selected_cities_list[i])) & (df['trg'].isin(selected_cities_list[i]))]

        year, month, day = int(filename[3:-10]), int(filename[8:-7]), int(filename[11:-4])
        date = datetime(year, month, day)
        df['date'] = date   
        df['date'] = pd.to_datetime(df['date'])

        df = df.sort_values(by=['src', 'trg'])

        movement_df = pd.concat([movement_df, df], axis = 0)

    movement_df = movement_df.reset_index(drop=True)
    movement_dfs_list.append(movement_df)

/var/folders/7g/y1r4j5314tj34x7tn5g1cdg80000gn/T/ipykernel_81014/1267743914.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  movement_df = pd.concat([movement_df, df], axis = 0)


/var/folders/7g/y1r4j5314tj34x7tn5g1cdg80000gn/T/ipykernel_81014/1267743914.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  movement_df = pd.concat([movement_df, df], axis = 0)
/var/folders/7g/y1r4j5314tj34x7tn5g1cdg80000gn/T/ipykernel_81014/1267743914.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  movement_df = pd.concat([movement_df, df], axis = 0)
/var/folders/7g/y1r4j5314tj34x7tn5g1cdg80000gn/T/ipykernel_81014/1267743914.py:31: FutureWarning: The behavior of DataFrame concatenatio

In [559]:
movement_ita_df = movement_dfs_list[0]
movement_spa_df = movement_dfs_list[1]
movement_fra_df = movement_dfs_list[2]
movement_eng_df = movement_dfs_list[3]

# Indexing

In [560]:
selected_cities_ita = np.sort(selected_cities_ita)
selected_cities_spa = np.sort(selected_cities_spa)
selected_cities_fra = np.sort(selected_cities_fra)
selected_cities_eng = np.sort(selected_cities_eng)

mapping_ita = {city: i for i, city in enumerate(selected_cities_ita, start=1)}
mapping_spa = {city: i for i, city in enumerate(selected_cities_spa, start=1)}
mapping_fra = {city: i for i, city in enumerate(selected_cities_fra, start=1)}
mapping_eng = {city: i for i, city in enumerate(selected_cities_eng, start=1)}

In [561]:
movement_ita_df['src'] = movement_ita_df['src'].map(mapping_ita)
movement_ita_df['trg'] = movement_ita_df['trg'].map(mapping_ita)
cases_ita_df['name'] = cases_ita_df['name'].map(mapping_ita)
cases_ita_df = cases_ita_df.sort_values(by=['name'])
cases_ita_df = cases_ita_df.reset_index(drop=True)

movement_spa_df['src'] = movement_spa_df['src'].map(mapping_spa)
movement_spa_df['trg'] = movement_spa_df['trg'].map(mapping_spa)
cases_spa_df['name'] = cases_spa_df['name'].map(mapping_spa)
cases_spa_df = cases_spa_df.sort_values(by=['name'])
cases_spa_df = cases_spa_df.reset_index(drop=True)

movement_fra_df['src'] = movement_fra_df['src'].map(mapping_fra)
movement_fra_df['trg'] = movement_fra_df['trg'].map(mapping_fra)
cases_fra_df['name'] = cases_fra_df['name'].map(mapping_fra)
cases_fra_df = cases_fra_df.sort_values(by=['name'])
cases_fra_df = cases_fra_df.reset_index(drop=True)

movement_eng_df['src'] = movement_eng_df['src'].map(mapping_eng)
movement_eng_df['trg'] = movement_eng_df['trg'].map(mapping_eng)
cases_eng_df['name'] = cases_eng_df['name'].map(mapping_eng)
cases_eng_df = cases_eng_df.sort_values(by=['name'])
cases_eng_df = cases_eng_df.reset_index(drop=True)

# Tests

## Movement-Cases Correlation

In [562]:
in_movement_ita_df = movement_ita_df.groupby(['trg', 'date'])['movement'].sum().reset_index()
in_movement_ita_df = in_movement_ita_df.rename(columns={'trg':'city'})
out_movement_ita_df = movement_ita_df.groupby(['src', 'date'])['movement'].sum().reset_index()
out_movement_ita_df = out_movement_ita_df.rename(columns={'src':'city'})

in_movement_spa_df = movement_spa_df.groupby(['trg', 'date'])['movement'].sum().reset_index()
in_movement_spa_df = in_movement_spa_df.rename(columns={'trg':'city'})
out_movement_spa_df = movement_spa_df.groupby(['src', 'date'])['movement'].sum().reset_index()
out_movement_spa_df = out_movement_spa_df.rename(columns={'src':'city'})

in_movement_fra_df = movement_fra_df.groupby(['trg', 'date'])['movement'].sum().reset_index()
in_movement_fra_df = in_movement_fra_df.rename(columns={'trg':'city'})
out_movement_fra_df = movement_fra_df.groupby(['src', 'date'])['movement'].sum().reset_index()
out_movement_fra_df = out_movement_fra_df.rename(columns={'src':'city'})

in_movement_eng_df = movement_eng_df.groupby(['trg', 'date'])['movement'].sum().reset_index()
in_movement_eng_df = in_movement_eng_df.rename(columns={'trg':'city'})
out_movement_eng_df = movement_eng_df.groupby(['src', 'date'])['movement'].sum().reset_index()
out_movement_eng_df = out_movement_eng_df.rename(columns={'src':'city'})

net_movement_ita_df = in_movement_ita_df.merge(out_movement_ita_df, on=['city', 'date'], suffixes=['_inward', '_outward'])
net_movement_ita_df['movement_net_lag_0'] = net_movement_ita_df['movement_inward'] - net_movement_ita_df['movement_outward']
net_movement_ita_df = net_movement_ita_df.drop(columns=['movement_inward', 'movement_outward'])

net_movement_spa_df = in_movement_spa_df.merge(out_movement_spa_df, on=['city', 'date'], suffixes=['_inward', '_outward'])
net_movement_spa_df['movement_net_lag_0'] = net_movement_spa_df['movement_inward'] - net_movement_spa_df['movement_outward']
net_movement_spa_df = net_movement_spa_df.drop(columns=['movement_inward', 'movement_outward'])

net_movement_fra_df = in_movement_fra_df.merge(out_movement_fra_df, on=['city', 'date'], suffixes=['_inward', '_outward'])
net_movement_fra_df['movement_net_lag_0'] = net_movement_fra_df['movement_inward'] - net_movement_fra_df['movement_outward']
net_movement_fra_df = net_movement_fra_df.drop(columns=['movement_inward', 'movement_outward'])

net_movement_eng_df = in_movement_eng_df.merge(out_movement_eng_df, on=['city', 'date'], suffixes=['_inward', '_outward'])
net_movement_eng_df['movement_net_lag_0'] = net_movement_eng_df['movement_inward'] - net_movement_eng_df['movement_outward']
net_movement_eng_df = net_movement_eng_df.drop(columns=['movement_inward', 'movement_outward'])

In [563]:
in_movement_ita_df = in_movement_ita_df.rename(columns={'movement':'movement_net_lag_0'})
out_movement_ita_df = out_movement_ita_df.rename(columns={'movement':'movement_net_lag_0'})

in_movement_spa_df = in_movement_spa_df.rename(columns={'movement':'movement_net_lag_0'})
out_movement_spa_df = out_movement_spa_df.rename(columns={'movement':'movement_net_lag_0'})

in_movement_fra_df = in_movement_fra_df.rename(columns={'movement':'movement_net_lag_0'})
out_movement_fra_df = out_movement_fra_df.rename(columns={'movement':'movement_net_lag_0'})

in_movement_eng_df = in_movement_eng_df.rename(columns={'movement':'movement_net_lag_0'})
out_movement_eng_df = out_movement_eng_df.rename(columns={'movement':'movement_net_lag_0'})

In [578]:
#ALWAYS IN THIS ORDER: Ita, Spa, Fra, Eng
mov_dfs_list = [out_movement_ita_df, out_movement_spa_df, out_movement_fra_df, out_movement_eng_df]

#Lagging
for df in mov_dfs_list:
    for i in range(15):
        df[f'movement_net_lag_{i}'] = df.groupby('city')['movement_net_lag_0'].shift(i)

In [579]:
cities = range(1, 31)
lags = range(15)
countries = [0, 1, 2, 3]

corrs_dfs = []

for country in countries:
    movement_df = mov_dfs_list[country]
    cases_df = cases_dfs_list[country]
    
    corrs_df = pd.DataFrame(index=cities, columns=[f'lag_{lag}' for lag in lags])
    
    for city in cities:
        movement_city = movement_df[movement_df['city'] == city]
        case_city = cases_df[cases_df['name'] == city].iloc[0, 1:]  # Skip the 'name' column
        
        for lag in lags:
            lagged_mov_vals = movement_city[f'movement_net_lag_{lag}'].values
            mask = ~np.isnan(lagged_mov_vals)
            lagged_mov_vals = lagged_mov_vals[mask]
            
            # Fix lenghts
            min_length = min(len(lagged_mov_vals), len(case_city))
            case_city_lagged = case_city.values[len(case_city)-min_length:]
            
            correlation = pd.Series(lagged_mov_vals).corr(pd.Series(case_city_lagged))
            corrs_df.at[city, f'lag_{lag}'] = correlation
    
    corrs_dfs.append(corrs_df)

corrs_ita_df = corrs_dfs[0]
corrs_spa_df = corrs_dfs[1]
corrs_fra_df = corrs_dfs[2]
corrs_eng_df = corrs_dfs[3]

/Users/marco/Library/CloudStorage/Dropbox/University/Bachelor/Year 3/Bocconi (2)/Thesis (GNNs Lomele)/Experiment/temporal_edge_regression/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/marco/Library/CloudStorage/Dropbox/University/Bachelor/Year 3/Bocconi (2)/Thesis (GNNs Lomele)/Experiment/temporal_edge_regression/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [580]:
max_corr_ita_df = pd.DataFrame(index=corrs_ita_df.index, columns=['best_lag', 'corr_value', 'second_best_lag','second_best_corr_value'])
max_corr_spa_df = pd.DataFrame(index=corrs_spa_df.index, columns=['best_lag', 'corr_value', 'second_best_lag','second_best_corr_value'])
max_corr_fra_df = pd.DataFrame(index=corrs_fra_df.index, columns=['best_lag', 'corr_value', 'second_best_lag','second_best_corr_value'])
max_corr_eng_df = pd.DataFrame(index=corrs_eng_df.index, columns=['best_lag', 'corr_value', 'second_best_lag','second_best_corr_value'])

max_corr_dfs_list = [max_corr_ita_df, max_corr_spa_df, max_corr_fra_df, max_corr_eng_df]
corrs_dfs_list = [corrs_ita_df, corrs_spa_df, corrs_fra_df, corrs_eng_df]

In [581]:
for country_idx in range(4):
    corrs_df = corrs_dfs_list[country_idx]
    max_corr = max_corr_dfs_list[country_idx]
    
    for city in corrs_df.index:
        value = corrs_df.loc[city].abs()

        top_values = value.sort_values(ascending=False).head(2)
        best_lag = top_values.index[0] if len(top_values) > 0 else None
        best_value = top_values.iloc[0] if len(top_values) > 0 else None
        second_best_lag = top_values.index[1] if len(top_values) > 1 else None
        second_best_value = top_values.iloc[1] if len(top_values) > 1 else None
        
        max_corr.at[city, 'best_lag'] = best_lag
        max_corr.at[city, 'corr_value'] = best_value
        max_corr.at[city, 'second_best_lag'] = second_best_lag
        max_corr.at[city, 'second_best_corr_value'] = second_best_value

In [585]:
max_corr_spa_df['best_lag'].value_counts()

best_lag
lag_5     4
lag_14    3
lag_12    3
lag_10    3
lag_0     3
lag_9     3
lag_1     3
lag_11    2
lag_7     1
lag_3     1
lag_6     1
lag_13    1
lag_2     1
lag_4     1
Name: count, dtype: int64

Next up:

1. Find optimal lag of movement data. Comapare correlation between number of covid cases today and movemement data lagged by 1 to 14 days. 
2. Implications of using directed vs undirected edges. 
3. Exploration of using features on the node that provide more information about COVID transmission: density of population, number of cities/towns.